# Mount ADSL Gen 2 storage to Databricks

At first you need to create an App in Azure Active Directory 
From overview you should take:

- Application (client) ID
- Directory (tenant) ID

Create a Client secrets on Certificates & Secrets of your App. Save generated secret value.

Generate token in Databricks User Settings and get its URL (https://adb-###############.0.azuredatabricks.net/?o=###############)

In Azure CLI run:

     pip install databricks-cli

     databricks configure --token
     > Databricks Host (should begin with https://): <databricks-url>
     > Token: <databricks-token>

After that create a scope and enter data from above:
  
    databricks secrets create-scope --scope <scope-name> --initial-manage-principal users
    databricks secrets put --scope <scope-name> --key SPID --string-value <application-client-id>
    databricks secrets put --scope <scope-name> --key SPKey --string-value <secret-app-value>
    databricks secrets put --scope <scope-name> --key DirectoryID --string-value <directory-tenant-id>
  
After that you can proceed further

In [0]:
ServicePrincipalID = dbutils.secrets.get(scope="data-bricks", key="SPID")
ServicePrincipalKey = dbutils.secrets.get(scope="data-bricks", key="SPKey")
DirectoryID = dbutils.secrets.get(scope="data-bricks", key="DirectoryID")

Directory = f"https://login.microsoftonline.com/{DirectoryID}/oauth2/token"

container_name = 'my-csv'
storage_account_name = 'mainstorageaccountv2'

url=f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net'
mnt_path = '/mnt/test'

configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": ServicePrincipalID,
  "fs.azure.account.oauth2.client.secret": ServicePrincipalKey,
  "fs.azure.account.oauth2.client.endpoint": Directory,
  "fs.azure.createRemoteFileSystemDuringInitialization": "false"
}

dbutils.fs.mount(
  source = url,
  mount_point = mnt_path,
  extra_configs = configs
)

Out[1]: True

### List mount

In [0]:
dbutils.fs.ls("/mnt/test") 

Out[2]: [FileInfo(path='dbfs:/mnt/test/dataout/', name='dataout/', size=0),
 FileInfo(path='dbfs:/mnt/test/market_data.csv', name='market_data.csv', size=68870)]

### Read file from storage

In [0]:
df = spark.read.format('csv').option("inferSchema", "true").option("header", "true").load("/mnt/test/market_data.csv")
df.head(5)

Out[7]: [Row(Date='16-Feb-16', EPAM=60.44, XOM=81.22, VIX=24.11, AAPL=96.639999, FB=101.610001, AMJ=23.44, GOOG=717.640015, ICHGF=33.0),
 Row(Date='12-Feb-16', EPAM=57.0, XOM=81.03, VIX=25.4, AAPL=93.989998, FB=102.010002, AMJ=22.05, GOOG=706.890015, ICHGF=33.0),
 Row(Date='11-Feb-16', EPAM=58.05, XOM=79.6, VIX=28.14, AAPL=93.699997, FB=101.910004, AMJ=20.88, GOOG=706.359985, ICHGF=33.0),
 Row(Date='10-Feb-16', EPAM=58.06, XOM=79.35, VIX=26.29, AAPL=94.269997, FB=101.0, AMJ=21.84, GOOG=706.849976, ICHGF=33.0),
 Row(Date='9-Feb-16', EPAM=58.69, XOM=80.08, VIX=26.54, AAPL=94.989998, FB=99.540001, AMJ=21.68, GOOG=701.02002, ICHGF=36.790001)]

### Unmount

In [0]:
dbutils.fs.unmount("/mnt/test") 

/mnt/test has been unmounted.
Out[8]: True

### The end